In [1]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/explore_img/explore_data.csv")
    shuffled_df = df.sample(len(df))

    for entry in shuffled_df.values:

        entry_img = cv2.imread(f"classification_data/explore_img/{entry[0]}")

        prepared_y = np.delete(entry,0) * 3

        if prepared_y[0] <= 960:
            prepared_y[0] =  -(960 - prepared_y[0])
        else:
            prepared_y[0] = prepared_y[0]-960
            
        if prepared_y[1] <= 540:
            prepared_y[1] = - (540 - prepared_y[1])
        else:
            prepared_y[1] = prepared_y[1] - 540
 
        X.append(entry_img)
        y.append(prepared_y)
        
    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [9]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    
    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.experimental.output_all_intermediates(True)
    return model

model = compile_model()

In [3]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)


(956, 360, 640, 3) (956, 2)


In [ ]:

model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val),batch_size = 24)


In [ ]:
model.save('ME_gpu.h5')

In [4]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.experimental.output_all_intermediates(True)
model_rap = tf.keras.models.load_model('ME_gpu.h5')
model_rap.compile(loss='mean_squared_error', optimizer=Adam())

for n in range(20):
    n = n +20
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    
    if x_real <= 960:
        x_real =  960 + x_real
    else:
        x_real = x_real + 960
        
    if y_real <= 540:
        y_real = 540 + y_real 
    else:
        y_real = y_real + 540
    
    img = np.asarray(X_val[n])

    tbp = np.expand_dims(img, axis=0)
    img = cv2.resize(img,(1920,1080))
    predictions = model_rap.predict(tbp)
    x, y = int(predictions[0][0]), int(predictions[0][1])
    
    if x <= 960:
        x =  960 + x
    else:
        x = x + 960
        
    if y <= 540:
        y = 540 + y 
    else:
        y = y + 540
    
    print(x, y, "\n",x_real, y_real)
    
    cv2.circle(img, (x,y), 15, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 10, (255,255,255), -1)
    
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

C:\Users\ukrst\.conda\envs\tf_6\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


1095 99 
 1080 93
1494 681 
 1494 732
561 70 
 546 69
395 313 
 354 303
518 256 
 480 264
529 607 
 372 285
918 893 
 900 912
1015 482 
 1338 807
325 326 
 285 297
810 150 
 801 150
803 345 
 783 357
1388 447 
 1125 927
1109 216 
 1110 231
1584 451 
 1575 444
970 48 
 942 54
1137 81 
 1140 93
1427 351 
 1467 342
1467 794 
 1356 924
592 808 
 579 831
1512 118 
 1569 138


In [ ]:
model_rap = tf.keras.models.load_model('mouse_explore_fhd_normalized_gpu.h5')
model_rap.compile(optimizer='adam',
              loss='mean_squared_error')


model_rap.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val),batch_size = 16)
model_rap.save('mouse_explore_fhd_normalized2.h5')